# 🎯 БЛОК 3 — ML ENGINEERING (Н)

## Fraud Detection Model Development

Этот notebook содержит полный пайплайн разработки ML-модели для обнаружения мошеннических транзакций:

1. **Baseline** (37-40) - Logistic Regression как базовая модель
2. **Model Development** (41-46) - Сравнение RandomForest, XGBoost, LightGBM, CatBoost
3. **Optimization** (47-51) - Подбор гиперпараметров, feature selection, CV, threshold optimization
4. **Interpretability** (52-56) - SHAP анализ для объяснения модели
5. **Model Packaging** (57-59) - Сохранение модели и документация

**Главная метрика:** F2-score (фокус на Recall - не пропустить мошенника)


In [ ]:
# Импорты
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
warnings.filterwarnings('ignore')

# ML библиотеки
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    precision_score, recall_score, f1_score, fbeta_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)
from sklearn.preprocessing import StandardScaler
import joblib

# Gradient Boosting
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# Оптимизация
import optuna

# Интерпретируемость
import shap

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Все библиотеки загружены успешно!")


In [ ]:
# Загрузка данных
DATA_PATH = '../data/processed/transactions_with_features.csv'

df = pd.read_csv(DATA_PATH)
print(f"✅ Данные загружены: {df.shape}")
print(f"\nКолонки: {len(df.columns)}")
print(f"Первые 5 колонок: {list(df.columns[:5])}")

# Проверка целевой переменной
if 'target' in df.columns:
    print(f"\n📊 Распределение классов:")
    print(df['target'].value_counts())
    imbalance_ratio = df['target'].value_counts()[0] / df['target'].value_counts()[1]
    print(f"\nДисбаланс: {imbalance_ratio:.2f}:1")
else:
    raise ValueError("❌ Колонка 'target' не найдена!")

# Разделение на признаки и целевую переменную
X = df.drop(columns=['target'], errors='ignore')
y = df['target']

# Удаляем нечисловые колонки (если есть)
non_numeric_cols = X.select_dtypes(include=['object']).columns
if len(non_numeric_cols) > 0:
    print(f"\n⚠️ Удаляем нечисловые колонки: {list(non_numeric_cols)}")
    X = X.select_dtypes(include=[np.number])

print(f"\n✅ Финальный размер признаков: {X.shape}")
print(f"Количество признаков: {X.shape[1]}")


## 📊 Baseline Model (Задачи 37-40)

### Задача 37: Train/Test Split (Stratified)
### Задача 38: Baseline модель Logistic Regression
### Задача 39: Метрики Precision, Recall, F2
### Задача 40: Confusion Matrix


In [ ]:
# Задача 37: Train/Test Split (Stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y  # Сохраняем пропорции классов
)

print(f"✅ Train/Test Split выполнен:")
print(f"  Train: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  Test: {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"\nРаспределение классов в train:")
print(y_train.value_counts())
print(f"\nРаспределение классов в test:")
print(y_test.value_counts())


In [ ]:
# Задача 38: Baseline модель Logistic Regression
print("--- Обучение Baseline модели (Logistic Regression) ---")

# Масштабирование для Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение с учетом дисбаланса классов
baseline_model = LogisticRegression(
    random_state=42,
    max_iter=1000,
    class_weight='balanced'  # Автоматическая балансировка классов
)

baseline_model.fit(X_train_scaled, y_train)

# Предсказания
y_pred_baseline = baseline_model.predict(X_test_scaled)
y_pred_proba_baseline = baseline_model.predict_proba(X_test_scaled)[:, 1]

print("✅ Baseline модель обучена!")


In [ ]:
# Задача 39: Метрики Precision, Recall, F2
def calculate_metrics(y_true, y_pred, y_pred_proba=None, model_name=""):
    """Вычисляет все необходимые метрики"""
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    f2 = fbeta_score(y_true, y_pred, beta=2)  # F2-score (больший вес Recall)
    
    metrics = {
        'Model': model_name,
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1,
        'F2-score': f2
    }
    
    if y_pred_proba is not None:
        roc_auc = roc_auc_score(y_true, y_pred_proba)
        metrics['ROC-AUC'] = roc_auc
    
    return metrics

# Метрики для Baseline
baseline_metrics = calculate_metrics(
    y_test, y_pred_baseline, y_pred_proba_baseline, "Baseline (Logistic Regression)"
)

print("📊 МЕТРИКИ BASELINE МОДЕЛИ:")
print("=" * 60)
for key, value in baseline_metrics.items():
    if key != 'Model':
        print(f"{key:15s}: {value:.4f}")
print("=" * 60)


In [ ]:
# Задача 40: Confusion Matrix
# Создаем директорию для сохранения графиков
os.makedirs('../model', exist_ok=True)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion Matrix
cm_baseline = confusion_matrix(y_test, y_pred_baseline)
sns.heatmap(cm_baseline, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Нормальные', 'Мошеннические'],
            yticklabels=['Нормальные', 'Мошеннические'])
axes[0].set_title('Confusion Matrix - Baseline (Logistic Regression)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Истинные значения')
axes[0].set_xlabel('Предсказанные значения')

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba_baseline)
axes[1].plot(fpr, tpr, label=f'Baseline (AUC = {baseline_metrics["ROC-AUC"]:.3f})', linewidth=2)
axes[1].plot([0, 1], [0, 1], 'k--', label='Random Classifier')
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curve - Baseline', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../model/baseline_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Baseline модель готова! Метрики сохранены.")


## 🚀 Model Development (Задачи 41-46)

### Задачи 41-44: Обучение моделей (RandomForest, XGBoost, LightGBM, CatBoost)
### Задача 45: Сравнение моделей по метрикам
### Задача 46: Выбор лучшей модели


In [ ]:
# Задача 41: RandomForest
print("--- Обучение RandomForest ---")
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
y_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]
print("✅ RandomForest обучен!")


In [ ]:
# Задача 42: XGBoost
print("--- Обучение XGBoost ---")
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    scale_pos_weight=len(y_train[y_train==0]) / len(y_train[y_train==1]),  # Балансировка
    random_state=42,
    eval_metric='logloss',
    n_jobs=-1
)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]
print("✅ XGBoost обучен!")


In [ ]:
# Задача 43: LightGBM
print("--- Обучение LightGBM ---")
lgb_model = lgb.LGBMClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=-1
)
lgb_model.fit(X_train, y_train)
y_pred_lgb = lgb_model.predict(X_test)
y_pred_proba_lgb = lgb_model.predict_proba(X_test)[:, 1]
print("✅ LightGBM обучен!")


In [ ]:
# Задача 44: CatBoost
print("--- Обучение CatBoost ---")
cb_model = cb.CatBoostClassifier(
    iterations=100,
    depth=6,
    learning_rate=0.1,
    class_weights=[1, len(y_train[y_train==0]) / len(y_train[y_train==1])],  # Балансировка
    random_state=42,
    verbose=False
)
cb_model.fit(X_train, y_train)
y_pred_cb = cb_model.predict(X_test)
y_pred_proba_cb = cb_model.predict_proba(X_test)[:, 1]
print("✅ CatBoost обучен!")


In [ ]:
# Задача 45: Сравнение моделей по метрикам
all_models = {
    'Baseline (Logistic Regression)': (y_pred_baseline, y_pred_proba_baseline),
    'RandomForest': (y_pred_rf, y_pred_proba_rf),
    'XGBoost': (y_pred_xgb, y_pred_proba_xgb),
    'LightGBM': (y_pred_lgb, y_pred_proba_lgb),
    'CatBoost': (y_pred_cb, y_pred_proba_cb)
}

results = []
for name, (y_pred, y_pred_proba) in all_models.items():
    metrics = calculate_metrics(y_test, y_pred, y_pred_proba, name)
    results.append(metrics)

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('F2-score', ascending=False)

print("📊 СРАВНЕНИЕ МОДЕЛЕЙ:")
print("=" * 80)
print(results_df.to_string(index=False))
print("=" * 80)

# Визуализация сравнения
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

metrics_to_plot = ['Precision', 'Recall', 'F1-score', 'F2-score']
for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx // 2, idx % 2]
    bars = ax.barh(results_df['Model'], results_df[metric], color=sns.color_palette("husl", len(results_df)))
    ax.set_xlabel(metric, fontsize=11, fontweight='bold')
    ax.set_title(f'{metric} по моделям', fontsize=12, fontweight='bold')
    ax.grid(alpha=0.3, axis='x')
    
    # Добавляем значения на столбцы
    for i, (bar, value) in enumerate(zip(bars, results_df[metric])):
        ax.text(value, i, f' {value:.4f}', va='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../model/models_comparison.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Задача 46: Выбор лучшей модели
best_model_name = results_df.iloc[0]['Model']
best_f2_score = results_df.iloc[0]['F2-score']

print(f"🏆 ЛУЧШАЯ МОДЕЛЬ: {best_model_name}")
print(f"   F2-score: {best_f2_score:.4f}")
print(f"\n📋 Все метрики лучшей модели:")
best_metrics = results_df[results_df['Model'] == best_model_name].iloc[0]
for metric in ['Precision', 'Recall', 'F1-score', 'F2-score', 'ROC-AUC']:
    if metric in best_metrics:
        print(f"   {metric}: {best_metrics[metric]:.4f}")

# Сохраняем ссылку на лучшую модель
if best_model_name == 'Baseline (Logistic Regression)':
    best_model = baseline_model
    best_scaler = scaler
elif best_model_name == 'RandomForest':
    best_model = rf_model
    best_scaler = None
elif best_model_name == 'XGBoost':
    best_model = xgb_model
    best_scaler = None
elif best_model_name == 'LightGBM':
    best_model = lgb_model
    best_scaler = None
elif best_model_name == 'CatBoost':
    best_model = cb_model
    best_scaler = None

print(f"\n✅ Лучшая модель выбрана: {best_model_name}")


## ⚙️ Optimization (Задачи 47-51)

### Задача 47: Подбор гиперпараметров (Optuna)
### Задача 48: Улучшение признаков (отбор feature importance)
### Задача 49: Cross-validation
### Задача 50: Оптимизация порога классификации
### Задача 51: Итоговые метрики


In [ ]:
# Задача 47: Подбор гиперпараметров (Optuna)
print("--- Оптимизация гиперпараметров с Optuna ---")
print("⚠️ Это может занять некоторое время...")

# Определяем, какую модель оптимизировать (берем лучшую из предыдущего шага)
# Для примера оптимизируем LightGBM (обычно показывает хорошие результаты)

def objective(trial):
    """Objective функция для Optuna"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'class_weight': 'balanced',
        'random_state': 42,
        'n_jobs': -1,
        'verbose': -1
    }
    
    model = lgb.LGBMClassifier(**params)
    
    # Используем cross-validation для оценки
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, 
                            scoring='fbeta', beta=2, n_jobs=-1)
    
    return scores.mean()

# Оптимизация (ограничиваем количество trials для скорости)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20, show_progress_bar=True)

print(f"\n✅ Оптимизация завершена!")
print(f"Лучший F2-score (CV): {study.best_value:.4f}")
print(f"Лучшие параметры:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# Обучаем модель с лучшими параметрами
best_params = study.best_params.copy()
best_params.update({
    'class_weight': 'balanced',
    'random_state': 42,
    'n_jobs': -1,
    'verbose': -1
})

optimized_model = lgb.LGBMClassifier(**best_params)
optimized_model.fit(X_train, y_train)
y_pred_optimized = optimized_model.predict(X_test)
y_pred_proba_optimized = optimized_model.predict_proba(X_test)[:, 1]

optimized_metrics = calculate_metrics(
    y_test, y_pred_optimized, y_pred_proba_optimized, "LightGBM (Optimized)"
)
print(f"\n📊 Метрики оптимизированной модели:")
for key, value in optimized_metrics.items():
    if key != 'Model':
        print(f"  {key}: {value:.4f}")


In [ ]:
# Задача 48: Улучшение признаков (отбор feature importance)
print("--- Отбор признаков по важности ---")

# Используем feature importance из оптимизированной модели
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': optimized_model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nТоп-20 самых важных признаков:")
print(feature_importance.head(20).to_string(index=False))

# Выбираем признаки с важностью выше порога (например, топ 80%)
importance_threshold = feature_importance['importance'].quantile(0.2)
selected_features = feature_importance[feature_importance['importance'] >= importance_threshold]['feature'].tolist()

print(f"\n✅ Отобрано {len(selected_features)} признаков из {len(X_train.columns)}")
print(f"Порог важности: {importance_threshold:.6f}")

# Переобучаем модель на отобранных признаках
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

optimized_model_selected = lgb.LGBMClassifier(**best_params)
optimized_model_selected.fit(X_train_selected, y_train)
y_pred_selected = optimized_model_selected.predict(X_test_selected)
y_pred_proba_selected = optimized_model_selected.predict_proba(X_test_selected)[:, 1]

selected_metrics = calculate_metrics(
    y_test, y_pred_selected, y_pred_proba_selected, "LightGBM (Optimized + Feature Selection)"
)
print(f"\n📊 Метрики после отбора признаков:")
for key, value in selected_metrics.items():
    if key != 'Model':
        print(f"  {key}: {value:.4f}")

# Визуализация важности признаков
plt.figure(figsize=(10, 8))
top_features = feature_importance.head(20)
plt.barh(range(len(top_features)), top_features['importance'], color='steelblue')
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Feature Importance', fontsize=12, fontweight='bold')
plt.title('Top-20 Feature Importance', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig('../model/feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Задача 49: Cross-validation
print("--- Cross-Validation оценка ---")

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# CV для оптимизированной модели с отобранными признаками
cv_scores_f2 = cross_val_score(
    optimized_model_selected, X_train_selected, y_train, 
    cv=cv, scoring='fbeta', beta=2, n_jobs=-1
)
cv_scores_recall = cross_val_score(
    optimized_model_selected, X_train_selected, y_train,
    cv=cv, scoring='recall', n_jobs=-1
)
cv_scores_precision = cross_val_score(
    optimized_model_selected, X_train_selected, y_train,
    cv=cv, scoring='precision', n_jobs=-1
)

print(f"\n📊 Cross-Validation результаты (5-fold):")
print(f"  F2-score: {cv_scores_f2.mean():.4f} (+/- {cv_scores_f2.std() * 2:.4f})")
print(f"  Recall:   {cv_scores_recall.mean():.4f} (+/- {cv_scores_recall.std() * 2:.4f})")
print(f"  Precision: {cv_scores_precision.mean():.4f} (+/- {cv_scores_precision.std() * 2:.4f})")

# Визуализация CV результатов
fig, ax = plt.subplots(figsize=(10, 6))
cv_results = pd.DataFrame({
    'Fold': range(1, 6),
    'F2-score': cv_scores_f2,
    'Recall': cv_scores_recall,
    'Precision': cv_scores_precision
})
cv_results.plot(x='Fold', y=['F2-score', 'Recall', 'Precision'], 
                kind='bar', ax=ax, color=['steelblue', 'coral', 'green'])
ax.set_ylabel('Score', fontsize=11, fontweight='bold')
ax.set_title('Cross-Validation Results (5-fold)', fontsize=12, fontweight='bold')
ax.set_xticklabels(cv_results['Fold'], rotation=0)
ax.legend(loc='best')
ax.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('../model/cv_results.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Задача 50: Оптимизация порога классификации
print("--- Оптимизация порога классификации ---")

# Пробуем разные пороги и выбираем тот, который максимизирует F2-score
thresholds = np.arange(0.1, 0.9, 0.05)
f2_scores = []
recall_scores = []
precision_scores = []

for threshold in thresholds:
    y_pred_thresh = (y_pred_proba_selected >= threshold).astype(int)
    f2 = fbeta_score(y_test, y_pred_thresh, beta=2)
    rec = recall_score(y_test, y_pred_thresh)
    prec = precision_score(y_test, y_pred_thresh)
    
    f2_scores.append(f2)
    recall_scores.append(rec)
    precision_scores.append(prec)

# Находим оптимальный порог
best_threshold_idx = np.argmax(f2_scores)
best_threshold = thresholds[best_threshold_idx]
best_f2_thresh = f2_scores[best_threshold_idx]

print(f"\n✅ Оптимальный порог: {best_threshold:.3f}")
print(f"   F2-score при этом пороге: {best_f2_thresh:.4f}")

# Применяем оптимальный порог
y_pred_optimal = (y_pred_proba_selected >= best_threshold).astype(int)
optimal_metrics = calculate_metrics(
    y_test, y_pred_optimal, y_pred_proba_selected, 
    f"LightGBM (Optimized + Feature Selection + Optimal Threshold={best_threshold:.3f})"
)

print(f"\n📊 Метрики с оптимальным порогом:")
for key, value in optimal_metrics.items():
    if key != 'Model':
        print(f"  {key}: {value:.4f}")

# Визуализация
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(thresholds, f2_scores, label='F2-score', linewidth=2, marker='o')
ax.plot(thresholds, recall_scores, label='Recall', linewidth=2, marker='s')
ax.plot(thresholds, precision_scores, label='Precision', linewidth=2, marker='^')
ax.axvline(best_threshold, color='red', linestyle='--', 
           label=f'Optimal Threshold = {best_threshold:.3f}')
ax.set_xlabel('Threshold', fontsize=11, fontweight='bold')
ax.set_ylabel('Score', fontsize=11, fontweight='bold')
ax.set_title('Threshold Optimization', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../model/threshold_optimization.png', dpi=300, bbox_inches='tight')
plt.show()

# Обновляем финальную модель
final_model = optimized_model_selected
final_threshold = best_threshold


In [ ]:
# Задача 51: Итоговые метрики
print("=" * 80)
print("📊 ИТОГОВЫЕ МЕТРИКИ ФИНАЛЬНОЙ МОДЕЛИ")
print("=" * 80)

final_metrics = optimal_metrics
for key, value in final_metrics.items():
    if key != 'Model':
        print(f"{key:15s}: {value:.4f}")

# Confusion Matrix финальной модели
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

cm_final = confusion_matrix(y_test, y_pred_optimal)
sns.heatmap(cm_final, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Нормальные', 'Мошеннические'],
            yticklabels=['Нормальные', 'Мошеннические'])
axes[0].set_title('Confusion Matrix - Final Model', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Истинные значения')
axes[0].set_xlabel('Предсказанные значения')

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba_selected)
axes[1].plot(fpr, tpr, label=f'Final Model (AUC = {final_metrics["ROC-AUC"]:.3f})', 
             linewidth=2, color='steelblue')
axes[1].plot([0, 1], [0, 1], 'k--', label='Random Classifier')
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curve - Final Model', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../model/final_model_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Итоговые метрики готовы!")


## 🔍 Interpretability (Задачи 52-56)

### Задача 52: Установить SHAP
### Задача 53: SHAP summary plot
### Задача 54: SHAP bar importance
### Задача 55: Пример объяснения конкретной транзакции
### Задача 56: Сохранить SHAP графики


In [ ]:
# Задача 52: Установить SHAP (уже установлен в requirements.txt)
print("--- SHAP Analysis ---")
print("✅ SHAP установлен и готов к использованию")

# Инициализируем SHAP explainer
# Для LightGBM используем TreeExplainer (быстрее и точнее)
explainer = shap.TreeExplainer(final_model)
shap_values = explainer.shap_values(X_test_selected)

# Для бинарной классификации shap_values - это список [values_class_0, values_class_1]
# Нас интересует класс 1 (мошенничество)
if isinstance(shap_values, list):
    shap_values_fraud = shap_values[1]  # Значения для класса мошенничества
else:
    shap_values_fraud = shap_values

print(f"✅ SHAP значения вычислены для {len(X_test_selected)} тестовых образцов")


In [ ]:
# Задача 53: SHAP summary plot
print("--- Создание SHAP Summary Plot ---")

plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values_fraud, X_test_selected, 
                  feature_names=selected_features,
                  show=False, max_display=20)
plt.title('SHAP Summary Plot (Top 20 Features)', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('../model/shap_summary_plot.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ SHAP Summary Plot сохранен!")


In [ ]:
# Задача 54: SHAP bar importance
print("--- Создание SHAP Bar Plot ---")

plt.figure(figsize=(10, 8))
shap.plots.bar(shap_values_fraud, feature_names=selected_features, 
               max_display=20, show=False)
plt.title('SHAP Feature Importance (Bar Plot)', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('../model/shap_bar_plot.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ SHAP Bar Plot сохранен!")


In [ ]:
# Задача 55: Пример объяснения конкретной транзакции
print("--- Пример объяснения конкретной транзакции ---")

# Находим пример мошеннической транзакции (если есть)
fraud_indices = y_test[y_test == 1].index
if len(fraud_indices) > 0:
    # Берем первую мошенническую транзакцию из тестовой выборки
    example_idx = fraud_indices[0]
    example_idx_in_test = list(y_test.index).index(example_idx)
    
    print(f"\n📋 Пример транзакции #{example_idx_in_test}:")
    print(f"   Истинный класс: Мошенническая (1)")
    print(f"   Предсказанная вероятность: {y_pred_proba_selected[example_idx_in_test]:.4f}")
    print(f"   Предсказанный класс: {'Мошенническая' if y_pred_optimal[example_idx_in_test] == 1 else 'Нормальная'}")
    
    # Waterfall plot для конкретного примера
    plt.figure(figsize=(10, 8))
    shap.waterfall_plot(
        shap.Explanation(
            values=shap_values_fraud[example_idx_in_test],
            base_values=explainer.expected_value[1] if isinstance(explainer.expected_value, list) else explainer.expected_value,
            data=X_test_selected.iloc[example_idx_in_test].values,
            feature_names=selected_features
        ),
        max_display=15,
        show=False
    )
    plt.title(f'SHAP Waterfall Plot - Transaction #{example_idx_in_test}\n(Predicted: {y_pred_proba_selected[example_idx_in_test]:.4f})', 
              fontsize=12, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig('../model/shap_waterfall_example.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✅ Пример объяснения создан!")
else:
    print("⚠️ В тестовой выборке нет мошеннических транзакций для примера")
    
    # Берем транзакцию с высокой вероятностью мошенничества
    high_prob_idx = np.argmax(y_pred_proba_selected)
    print(f"\n📋 Пример транзакции с высокой вероятностью мошенничества:")
    print(f"   Индекс: {high_prob_idx}")
    print(f"   Предсказанная вероятность: {y_pred_proba_selected[high_prob_idx]:.4f}")
    
    plt.figure(figsize=(10, 8))
    shap.waterfall_plot(
        shap.Explanation(
            values=shap_values_fraud[high_prob_idx],
            base_values=explainer.expected_value[1] if isinstance(explainer.expected_value, list) else explainer.expected_value,
            data=X_test_selected.iloc[high_prob_idx].values,
            feature_names=selected_features
        ),
        max_display=15,
        show=False
    )
    plt.title(f'SHAP Waterfall Plot - High Risk Transaction\n(Predicted: {y_pred_proba_selected[high_prob_idx]:.4f})', 
              fontsize=12, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig('../model/shap_waterfall_example.png', dpi=300, bbox_inches='tight')
    plt.show()


In [ ]:
# Задача 56: Сохранить SHAP графики (уже сохранены выше)
print("✅ Все SHAP графики сохранены:")
print("  - shap_summary_plot.png")
print("  - shap_bar_plot.png")
print("  - shap_waterfall_example.png")


## 📦 Model Packaging (Задачи 57-59)

### Задача 57: Сохранить модель в model.pkl
### Задача 58: Документировать вход/выход модели
### Задача 59: Создать notebook 03_final_model.ipynb (✅ уже создан)


In [ ]:
# Задача 57: Сохранить модель в model.pkl
# Создаем директорию model если её нет
os.makedirs('../model', exist_ok=True)

# Сохраняем модель, список признаков и порог
model_package = {
    'model': final_model,
    'selected_features': selected_features,
    'threshold': final_threshold,
    'scaler': None,  # Не используется для LightGBM
    'model_type': 'LightGBM',
    'metrics': final_metrics
}

joblib.dump(model_package, '../model/model.pkl')
print("✅ Модель сохранена в ../model/model.pkl")

# Также сохраняем отдельно для удобства
joblib.dump(final_model, '../model/final_model.pkl')
joblib.dump(selected_features, '../model/selected_features.pkl')
joblib.dump(final_threshold, '../model/threshold.pkl')

print("✅ Дополнительные файлы сохранены:")
print("  - final_model.pkl")
print("  - selected_features.pkl")
print("  - threshold.pkl")


In [ ]:
# Задача 58: Документировать вход/выход модели
model_doc = f"""# 📋 Документация модели Fraud Detection

## Модель
- **Тип:** {model_package['model_type']}
- **Версия:** 1.0
- **Дата создания:** {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

## Метрики производительности
- **F2-score:** {final_metrics['F2-score']:.4f}
- **Recall:** {final_metrics['Recall']:.4f}
- **Precision:** {final_metrics['Precision']:.4f}
- **F1-score:** {final_metrics['F1-score']:.4f}
- **ROC-AUC:** {final_metrics['ROC-AUC']:.4f}

## Оптимальный порог классификации
- **Threshold:** {final_threshold:.3f}

## Входные данные (Input)
Модель принимает на вход DataFrame со следующими признаками ({len(selected_features)} признаков):

{chr(10).join([f"- {feat}" for feat in selected_features[:20]])}
... и еще {len(selected_features) - 20} признаков

**Требования:**
- Все признаки должны быть числовыми
- Пропущенные значения должны быть заполнены (рекомендуется 0)
- Признаки должны быть в том же порядке, что и в списке selected_features

## Выходные данные (Output)
Модель возвращает:
1. **Вероятность мошенничества** (float): значение от 0 до 1
   - Близко к 0: низкая вероятность мошенничества
   - Близко к 1: высокая вероятность мошенничества

2. **Класс** (int): 0 или 1
   - 0: Нормальная транзакция
   - 1: Мошенническая транзакция
   - Определяется сравнением вероятности с порогом ({final_threshold:.3f})

## Пример использования

```python
import joblib
import pandas as pd

# Загрузка модели
model_package = joblib.load('model/model.pkl')
model = model_package['model']
selected_features = model_package['selected_features']
threshold = model_package['threshold']

# Подготовка данных
# X_new должен быть DataFrame с признаками из selected_features
X_new = pd.DataFrame(...)  # ваши данные

# Предсказание
probabilities = model.predict_proba(X_new[selected_features])[:, 1]
predictions = (probabilities >= threshold).astype(int)

# Результаты
for i, (prob, pred) in enumerate(zip(probabilities, predictions)):
    print(f"Транзакция {{i}}: вероятность={{prob:.4f}}, класс={{'Мошенническая' if pred == 1 else 'Нормальная'}}")
```

## Важные замечания
1. Модель обучена на данных с дисбалансом классов (~78:1)
2. Главная метрика - F2-score (фокус на Recall - не пропустить мошенника)
3. Порог классификации оптимизирован для максимизации F2-score
4. Для интерпретации результатов используйте SHAP значения (см. графики в model/)
"""

# Сохраняем документацию
with open('../model/MODEL_DOCUMENTATION.md', 'w', encoding='utf-8') as f:
    f.write(model_doc)

print("✅ Документация сохранена в ../model/MODEL_DOCUMENTATION.md")
print("\n" + "="*80)
print(model_doc)
print("="*80)


## ✅ Итоги

Все задачи блока 3 (ML Engineering) выполнены:

- ✅ **Baseline** (37-40): Train/test split, Logistic Regression, метрики, confusion matrix
- ✅ **Model Development** (41-46): Обучены и сравнены RandomForest, XGBoost, LightGBM, CatBoost
- ✅ **Optimization** (47-51): Оптимизация гиперпараметров, feature selection, CV, threshold optimization
- ✅ **Interpretability** (52-56): SHAP анализ, графики важности признаков, примеры объяснений
- ✅ **Model Packaging** (57-59): Модель сохранена, документация создана

**Финальная модель:** LightGBM с оптимизированными гиперпараметрами и отобранными признаками

Модель готова к использованию в API!

> **Примечание:** Финальные метрики (F2-score, Recall, Precision, ROC-AUC) можно увидеть в ячейке "Задача 51: Итоговые метрики" выше.


In [ ]:
# Импорты
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ML библиотеки
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    precision_score, recall_score, f1_score, fbeta_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)
from sklearn.preprocessing import StandardScaler
import joblib

# Gradient Boosting
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# Оптимизация
import optuna

# Интерпретируемость
import shap

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Все библиотеки загружены успешно!")


In [ ]:
# Загрузка данных
DATA_PATH = '../data/processed/transactions_with_features.csv'

df = pd.read_csv(DATA_PATH)
print(f"✅ Данные загружены: {df.shape}")
print(f"\nКолонки: {len(df.columns)}")
print(f"Первые 5 колонок: {list(df.columns[:5])}")

# Проверка целевой переменной
if 'target' in df.columns:
    print(f"\n📊 Распределение классов:")
    print(df['target'].value_counts())
    print(f"\nДисбаланс: {df['target'].value_counts()[0] / df['target'].value_counts()[1]:.2f}:1")
else:
    raise ValueError("❌ Колонка 'target' не найдена!")

# Разделение на признаки и целевую переменную
X = df.drop(columns=['target'], errors='ignore')
y = df['target']

# Удаляем нечисловые колонки (если есть)
non_numeric_cols = X.select_dtypes(include=['object']).columns
if len(non_numeric_cols) > 0:
    print(f"\n⚠️ Удаляем нечисловые колонки: {list(non_numeric_cols)}")
    X = X.select_dtypes(include=[np.number])

print(f"\n✅ Финальный размер признаков: {X.shape}")
print(f"Количество признаков: {X.shape[1]}")


In [ ]:
# Задача 37: Train/Test Split (Stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y  # Сохраняем пропорции классов
)

print(f"✅ Train/Test Split выполнен:")
print(f"  Train: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  Test: {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"\nРаспределение классов в train:")
print(y_train.value_counts())
print(f"\nРаспределение классов в test:")
print(y_test.value_counts())


In [ ]:
# Задача 38: Baseline модель Logistic Regression
print("--- Обучение Baseline модели (Logistic Regression) ---")

# Масштабирование для Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение с учетом дисбаланса классов
baseline_model = LogisticRegression(
    random_state=42,
    max_iter=1000,
    class_weight='balanced'  # Автоматическая балансировка классов
)

baseline_model.fit(X_train_scaled, y_train)

# Предсказания
y_pred_baseline = baseline_model.predict(X_test_scaled)
y_pred_proba_baseline = baseline_model.predict_proba(X_test_scaled)[:, 1]

print("✅ Baseline модель обучена!")


In [ ]:
# Задача 39: Метрики Precision, Recall, F2
def calculate_metrics(y_true, y_pred, y_pred_proba=None, model_name=""):
    """Вычисляет все необходимые метрики"""
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    f2 = fbeta_score(y_true, y_pred, beta=2)  # F2-score (больший вес Recall)
    
    metrics = {
        'Model': model_name,
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1,
        'F2-score': f2
    }
    
    if y_pred_proba is not None:
        roc_auc = roc_auc_score(y_true, y_pred_proba)
        metrics['ROC-AUC'] = roc_auc
    
    return metrics

# Метрики для Baseline
baseline_metrics = calculate_metrics(
    y_test, y_pred_baseline, y_pred_proba_baseline, "Baseline (Logistic Regression)"
)

print("📊 МЕТРИКИ BASELINE МОДЕЛИ:")
print("=" * 60)
for key, value in baseline_metrics.items():
    if key != 'Model':
        print(f"{key:15s}: {value:.4f}")
print("=" * 60)


In [ ]:
# Задача 40: Confusion Matrix
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion Matrix
cm_baseline = confusion_matrix(y_test, y_pred_baseline)
sns.heatmap(cm_baseline, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Нормальные', 'Мошеннические'],
            yticklabels=['Нормальные', 'Мошеннические'])
axes[0].set_title('Confusion Matrix - Baseline (Logistic Regression)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Истинные значения')
axes[0].set_xlabel('Предсказанные значения')

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba_baseline)
axes[1].plot(fpr, tpr, label=f'Baseline (AUC = {baseline_metrics["ROC-AUC"]:.3f})', linewidth=2)
axes[1].plot([0, 1], [0, 1], 'k--', label='Random Classifier')
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curve - Baseline', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../model/baseline_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Baseline модель готова! Метрики сохранены.")


In [ ]:
# Задача 41: RandomForest
print("--- Обучение RandomForest ---")
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
y_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]
print("✅ RandomForest обучен!")


In [ ]:
# Задача 42: XGBoost
print("--- Обучение XGBoost ---")
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    scale_pos_weight=len(y_train[y_train==0]) / len(y_train[y_train==1]),  # Балансировка
    random_state=42,
    eval_metric='logloss',
    n_jobs=-1
)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]
print("✅ XGBoost обучен!")


In [ ]:
# Задача 43: LightGBM
print("--- Обучение LightGBM ---")
lgb_model = lgb.LGBMClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=-1
)
lgb_model.fit(X_train, y_train)
y_pred_lgb = lgb_model.predict(X_test)
y_pred_proba_lgb = lgb_model.predict_proba(X_test)[:, 1]
print("✅ LightGBM обучен!")


In [ ]:
# Задача 44: CatBoost
print("--- Обучение CatBoost ---")
cb_model = cb.CatBoostClassifier(
    iterations=100,
    depth=6,
    learning_rate=0.1,
    class_weights=[1, len(y_train[y_train==0]) / len(y_train[y_train==1])],  # Балансировка
    random_state=42,
    verbose=False
)
cb_model.fit(X_train, y_train)
y_pred_cb = cb_model.predict(X_test)
y_pred_proba_cb = cb_model.predict_proba(X_test)[:, 1]
print("✅ CatBoost обучен!")


In [ ]:
# Задача 45: Сравнение моделей по метрикам
all_models = {
    'Baseline (Logistic Regression)': (y_pred_baseline, y_pred_proba_baseline),
    'RandomForest': (y_pred_rf, y_pred_proba_rf),
    'XGBoost': (y_pred_xgb, y_pred_proba_xgb),
    'LightGBM': (y_pred_lgb, y_pred_proba_lgb),
    'CatBoost': (y_pred_cb, y_pred_proba_cb)
}

results = []
for name, (y_pred, y_pred_proba) in all_models.items():
    metrics = calculate_metrics(y_test, y_pred, y_pred_proba, name)
    results.append(metrics)

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('F2-score', ascending=False)

print("📊 СРАВНЕНИЕ МОДЕЛЕЙ:")
print("=" * 80)
print(results_df.to_string(index=False))
print("=" * 80)

# Визуализация сравнения
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

metrics_to_plot = ['Precision', 'Recall', 'F1-score', 'F2-score']
for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx // 2, idx % 2]
    bars = ax.barh(results_df['Model'], results_df[metric], color=sns.color_palette("husl", len(results_df)))
    ax.set_xlabel(metric, fontsize=11, fontweight='bold')
    ax.set_title(f'{metric} по моделям', fontsize=12, fontweight='bold')
    ax.grid(alpha=0.3, axis='x')
    
    # Добавляем значения на столбцы
    for i, (bar, value) in enumerate(zip(bars, results_df[metric])):
        ax.text(value, i, f' {value:.4f}', va='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../model/models_comparison.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Задача 46: Выбор лучшей модели
best_model_name = results_df.iloc[0]['Model']
best_f2_score = results_df.iloc[0]['F2-score']

print(f"🏆 ЛУЧШАЯ МОДЕЛЬ: {best_model_name}")
print(f"   F2-score: {best_f2_score:.4f}")
print(f"\n📋 Все метрики лучшей модели:")
best_metrics = results_df[results_df['Model'] == best_model_name].iloc[0]
for metric in ['Precision', 'Recall', 'F1-score', 'F2-score', 'ROC-AUC']:
    if metric in best_metrics:
        print(f"   {metric}: {best_metrics[metric]:.4f}")

# Сохраняем ссылку на лучшую модель
if best_model_name == 'Baseline (Logistic Regression)':
    best_model = baseline_model
    best_scaler = scaler
elif best_model_name == 'RandomForest':
    best_model = rf_model
    best_scaler = None
elif best_model_name == 'XGBoost':
    best_model = xgb_model
    best_scaler = None
elif best_model_name == 'LightGBM':
    best_model = lgb_model
    best_scaler = None
elif best_model_name == 'CatBoost':
    best_model = cb_model
    best_scaler = None

print(f"\n✅ Лучшая модель выбрана: {best_model_name}")


In [ ]:
# Задача 47: Подбор гиперпараметров (Optuna)
print("--- Оптимизация гиперпараметров с Optuna ---")
print("⚠️ Это может занять некоторое время...")

# Определяем, какую модель оптимизировать (берем лучшую из предыдущего шага)
# Для примера оптимизируем LightGBM (обычно показывает хорошие результаты)

def objective(trial):
    """Objective функция для Optuna"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'class_weight': 'balanced',
        'random_state': 42,
        'n_jobs': -1,
        'verbose': -1
    }
    
    model = lgb.LGBMClassifier(**params)
    
    # Используем cross-validation для оценки
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, 
                            scoring='fbeta', beta=2, n_jobs=-1)
    
    return scores.mean()

# Оптимизация (ограничиваем количество trials для скорости)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20, show_progress_bar=True)

print(f"\n✅ Оптимизация завершена!")
print(f"Лучший F2-score (CV): {study.best_value:.4f}")
print(f"Лучшие параметры:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# Обучаем модель с лучшими параметрами
best_params = study.best_params.copy()
best_params.update({
    'class_weight': 'balanced',
    'random_state': 42,
    'n_jobs': -1,
    'verbose': -1
})

optimized_model = lgb.LGBMClassifier(**best_params)
optimized_model.fit(X_train, y_train)
y_pred_optimized = optimized_model.predict(X_test)
y_pred_proba_optimized = optimized_model.predict_proba(X_test)[:, 1]

optimized_metrics = calculate_metrics(
    y_test, y_pred_optimized, y_pred_proba_optimized, "LightGBM (Optimized)"
)
print(f"\n📊 Метрики оптимизированной модели:")
for key, value in optimized_metrics.items():
    if key != 'Model':
        print(f"  {key}: {value:.4f}")


In [ ]:
# Задача 48: Улучшение признаков (отбор feature importance)
print("--- Отбор признаков по важности ---")

# Используем feature importance из оптимизированной модели
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': optimized_model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nТоп-20 самых важных признаков:")
print(feature_importance.head(20).to_string(index=False))

# Выбираем признаки с важностью выше порога (например, топ 80%)
importance_threshold = feature_importance['importance'].quantile(0.2)
selected_features = feature_importance[feature_importance['importance'] >= importance_threshold]['feature'].tolist()

print(f"\n✅ Отобрано {len(selected_features)} признаков из {len(X_train.columns)}")
print(f"Порог важности: {importance_threshold:.6f}")

# Переобучаем модель на отобранных признаках
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

optimized_model_selected = lgb.LGBMClassifier(**best_params)
optimized_model_selected.fit(X_train_selected, y_train)
y_pred_selected = optimized_model_selected.predict(X_test_selected)
y_pred_proba_selected = optimized_model_selected.predict_proba(X_test_selected)[:, 1]

selected_metrics = calculate_metrics(
    y_test, y_pred_selected, y_pred_proba_selected, "LightGBM (Optimized + Feature Selection)"
)
print(f"\n📊 Метрики после отбора признаков:")
for key, value in selected_metrics.items():
    if key != 'Model':
        print(f"  {key}: {value:.4f}")

# Визуализация важности признаков
plt.figure(figsize=(10, 8))
top_features = feature_importance.head(20)
plt.barh(range(len(top_features)), top_features['importance'], color='steelblue')
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Feature Importance', fontsize=12, fontweight='bold')
plt.title('Top-20 Feature Importance', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig('../model/feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Задача 49: Cross-validation
print("--- Cross-Validation оценка ---")

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# CV для оптимизированной модели с отобранными признаками
cv_scores_f2 = cross_val_score(
    optimized_model_selected, X_train_selected, y_train, 
    cv=cv, scoring='fbeta', beta=2, n_jobs=-1
)
cv_scores_recall = cross_val_score(
    optimized_model_selected, X_train_selected, y_train,
    cv=cv, scoring='recall', n_jobs=-1
)
cv_scores_precision = cross_val_score(
    optimized_model_selected, X_train_selected, y_train,
    cv=cv, scoring='precision', n_jobs=-1
)

print(f"\n📊 Cross-Validation результаты (5-fold):")
print(f"  F2-score: {cv_scores_f2.mean():.4f} (+/- {cv_scores_f2.std() * 2:.4f})")
print(f"  Recall:   {cv_scores_recall.mean():.4f} (+/- {cv_scores_recall.std() * 2:.4f})")
print(f"  Precision: {cv_scores_precision.mean():.4f} (+/- {cv_scores_precision.std() * 2:.4f})")

# Визуализация CV результатов
fig, ax = plt.subplots(figsize=(10, 6))
cv_results = pd.DataFrame({
    'Fold': range(1, 6),
    'F2-score': cv_scores_f2,
    'Recall': cv_scores_recall,
    'Precision': cv_scores_precision
})
cv_results.plot(x='Fold', y=['F2-score', 'Recall', 'Precision'], 
                kind='bar', ax=ax, color=['steelblue', 'coral', 'green'])
ax.set_ylabel('Score', fontsize=11, fontweight='bold')
ax.set_title('Cross-Validation Results (5-fold)', fontsize=12, fontweight='bold')
ax.set_xticklabels(cv_results['Fold'], rotation=0)
ax.legend(loc='best')
ax.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('../model/cv_results.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Задача 50: Оптимизация порога классификации
print("--- Оптимизация порога классификации ---")

# Пробуем разные пороги и выбираем тот, который максимизирует F2-score
thresholds = np.arange(0.1, 0.9, 0.05)
f2_scores = []
recall_scores = []
precision_scores = []

for threshold in thresholds:
    y_pred_thresh = (y_pred_proba_selected >= threshold).astype(int)
    f2 = fbeta_score(y_test, y_pred_thresh, beta=2)
    rec = recall_score(y_test, y_pred_thresh)
    prec = precision_score(y_test, y_pred_thresh)
    
    f2_scores.append(f2)
    recall_scores.append(rec)
    precision_scores.append(prec)

# Находим оптимальный порог
best_threshold_idx = np.argmax(f2_scores)
best_threshold = thresholds[best_threshold_idx]
best_f2_thresh = f2_scores[best_threshold_idx]

print(f"\n✅ Оптимальный порог: {best_threshold:.3f}")
print(f"   F2-score при этом пороге: {best_f2_thresh:.4f}")

# Применяем оптимальный порог
y_pred_optimal = (y_pred_proba_selected >= best_threshold).astype(int)
optimal_metrics = calculate_metrics(
    y_test, y_pred_optimal, y_pred_proba_selected, 
    f"LightGBM (Optimized + Feature Selection + Optimal Threshold={best_threshold:.3f})"
)

print(f"\n📊 Метрики с оптимальным порогом:")
for key, value in optimal_metrics.items():
    if key != 'Model':
        print(f"  {key}: {value:.4f}")

# Визуализация
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(thresholds, f2_scores, label='F2-score', linewidth=2, marker='o')
ax.plot(thresholds, recall_scores, label='Recall', linewidth=2, marker='s')
ax.plot(thresholds, precision_scores, label='Precision', linewidth=2, marker='^')
ax.axvline(best_threshold, color='red', linestyle='--', 
           label=f'Optimal Threshold = {best_threshold:.3f}')
ax.set_xlabel('Threshold', fontsize=11, fontweight='bold')
ax.set_ylabel('Score', fontsize=11, fontweight='bold')
ax.set_title('Threshold Optimization', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../model/threshold_optimization.png', dpi=300, bbox_inches='tight')
plt.show()

# Обновляем финальную модель
final_model = optimized_model_selected
final_threshold = best_threshold


In [ ]:
# Задача 51: Итоговые метрики
print("=" * 80)
print("📊 ИТОГОВЫЕ МЕТРИКИ ФИНАЛЬНОЙ МОДЕЛИ")
print("=" * 80)

final_metrics = optimal_metrics
for key, value in final_metrics.items():
    if key != 'Model':
        print(f"{key:15s}: {value:.4f}")

# Confusion Matrix финальной модели
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

cm_final = confusion_matrix(y_test, y_pred_optimal)
sns.heatmap(cm_final, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Нормальные', 'Мошеннические'],
            yticklabels=['Нормальные', 'Мошеннические'])
axes[0].set_title('Confusion Matrix - Final Model', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Истинные значения')
axes[0].set_xlabel('Предсказанные значения')

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba_selected)
axes[1].plot(fpr, tpr, label=f'Final Model (AUC = {final_metrics["ROC-AUC"]:.3f})', 
             linewidth=2, color='steelblue')
axes[1].plot([0, 1], [0, 1], 'k--', label='Random Classifier')
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curve - Final Model', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../model/final_model_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Итоговые метрики готовы!")


In [ ]:
# Задача 52: Установить SHAP (уже установлен в requirements.txt)
print("--- SHAP Analysis ---")
print("✅ SHAP установлен и готов к использованию")

# Инициализируем SHAP explainer
# Для LightGBM используем TreeExplainer (быстрее и точнее)
explainer = shap.TreeExplainer(final_model)
shap_values = explainer.shap_values(X_test_selected)

# Для бинарной классификации shap_values - это список [values_class_0, values_class_1]
# Нас интересует класс 1 (мошенничество)
if isinstance(shap_values, list):
    shap_values_fraud = shap_values[1]  # Значения для класса мошенничества
else:
    shap_values_fraud = shap_values

print(f"✅ SHAP значения вычислены для {len(X_test_selected)} тестовых образцов")


In [ ]:
# Задача 53: SHAP summary plot
print("--- Создание SHAP Summary Plot ---")

plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values_fraud, X_test_selected, 
                  feature_names=selected_features,
                  show=False, max_display=20)
plt.title('SHAP Summary Plot (Top 20 Features)', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('../model/shap_summary_plot.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ SHAP Summary Plot сохранен!")


In [ ]:
# Задача 54: SHAP bar importance
print("--- Создание SHAP Bar Plot ---")

plt.figure(figsize=(10, 8))
shap.plots.bar(shap_values_fraud, feature_names=selected_features, 
               max_display=20, show=False)
plt.title('SHAP Feature Importance (Bar Plot)', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('../model/shap_bar_plot.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ SHAP Bar Plot сохранен!")


In [ ]:
# Задача 55: Пример объяснения конкретной транзакции
print("--- Пример объяснения конкретной транзакции ---")

# Находим пример мошеннической транзакции (если есть)
fraud_indices = y_test[y_test == 1].index
if len(fraud_indices) > 0:
    # Берем первую мошенническую транзакцию из тестовой выборки
    example_idx = fraud_indices[0]
    example_idx_in_test = list(y_test.index).index(example_idx)
    
    print(f"\n📋 Пример транзакции #{example_idx_in_test}:")
    print(f"   Истинный класс: Мошенническая (1)")
    print(f"   Предсказанная вероятность: {y_pred_proba_selected[example_idx_in_test]:.4f}")
    print(f"   Предсказанный класс: {'Мошенническая' if y_pred_optimal[example_idx_in_test] == 1 else 'Нормальная'}")
    
    # Waterfall plot для конкретного примера
    plt.figure(figsize=(10, 8))
    shap.waterfall_plot(
        shap.Explanation(
            values=shap_values_fraud[example_idx_in_test],
            base_values=explainer.expected_value[1] if isinstance(explainer.expected_value, list) else explainer.expected_value,
            data=X_test_selected.iloc[example_idx_in_test].values,
            feature_names=selected_features
        ),
        max_display=15,
        show=False
    )
    plt.title(f'SHAP Waterfall Plot - Transaction #{example_idx_in_test}\n(Predicted: {y_pred_proba_selected[example_idx_in_test]:.4f})', 
              fontsize=12, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig('../model/shap_waterfall_example.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✅ Пример объяснения создан!")
else:
    print("⚠️ В тестовой выборке нет мошеннических транзакций для примера")
    
    # Берем транзакцию с высокой вероятностью мошенничества
    high_prob_idx = np.argmax(y_pred_proba_selected)
    print(f"\n📋 Пример транзакции с высокой вероятностью мошенничества:")
    print(f"   Индекс: {high_prob_idx}")
    print(f"   Предсказанная вероятность: {y_pred_proba_selected[high_prob_idx]:.4f}")
    
    plt.figure(figsize=(10, 8))
    shap.waterfall_plot(
        shap.Explanation(
            values=shap_values_fraud[high_prob_idx],
            base_values=explainer.expected_value[1] if isinstance(explainer.expected_value, list) else explainer.expected_value,
            data=X_test_selected.iloc[high_prob_idx].values,
            feature_names=selected_features
        ),
        max_display=15,
        show=False
    )
    plt.title(f'SHAP Waterfall Plot - High Risk Transaction\n(Predicted: {y_pred_proba_selected[high_prob_idx]:.4f})', 
              fontsize=12, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig('../model/shap_waterfall_example.png', dpi=300, bbox_inches='tight')
    plt.show()


In [ ]:
# Задача 56: Сохранить SHAP графики (уже сохранены выше)
print("✅ Все SHAP графики сохранены:")
print("  - shap_summary_plot.png")
print("  - shap_bar_plot.png")
print("  - shap_waterfall_example.png")


In [ ]:
# Задача 57: Сохранить модель в model.pkl
import os

# Создаем директорию model если её нет
os.makedirs('../model', exist_ok=True)

# Сохраняем модель, список признаков и порог
model_package = {
    'model': final_model,
    'selected_features': selected_features,
    'threshold': final_threshold,
    'scaler': None,  # Не используется для LightGBM
    'model_type': 'LightGBM',
    'metrics': final_metrics
}

joblib.dump(model_package, '../model/model.pkl')
print("✅ Модель сохранена в ../model/model.pkl")

# Также сохраняем отдельно для удобства
joblib.dump(final_model, '../model/final_model.pkl')
joblib.dump(selected_features, '../model/selected_features.pkl')
joblib.dump(final_threshold, '../model/threshold.pkl')

print("✅ Дополнительные файлы сохранены:")
print("  - final_model.pkl")
print("  - selected_features.pkl")
print("  - threshold.pkl")
